# **Regression Diagnostics of Multiple Linear Regression using the Example of the Dataset "*cacao.csv*"**

## Installing and Loading Libraries

Before we can begin with multiple linear regression, we need to install necessary libraries. Libraries are loaded as needed.

First, we update the package installer pip and then install the libraries numpy, pandas, matplotlib, scikit-learn, seaborn, and statsmodels.

Finally, we load the libraries so they can be used.

In [8]:
# Update package installer

!python -m pip install --upgrade pip

# Install libraries

%pip install numpy >nul & pip install pandas >nul & pip install matplotlib >nul & pip install scikit-learn >nul & pip install seaborn >nul & pip install statsmodels >nul & pip install requests >nul

Note: you may need to restart the kernel to use updated packages.


In [15]:
# Reset environment

%reset -f

## Building the Model

We do not want to optimize the model at this point; instead, we want to take a closer look at the diagnostics that need to be examined and interpret the results of the model diagnostics.

First, we need to create the model and load the data.

For this, we use "stem_diameter" as the dependent variable and all other variables as predictors.

In [17]:
import pandas as pd
import statsmodels.api as sm

url = "https://raw.githubusercontent.com/reyar/Statsomat/master/cacao.csv"
df = pd.read_csv(url)

y = df['stem_diameter']
x = df.drop('stem_diameter', axis=1) # axis = 1 to look at columns instead of rows
x = sm.add_constant(x)
model = sm.OLS(y, x).fit() # train the model


# **Regression Diagnostics**

The model was developed under certain assumptions. To examine these, we conduct specific diagnostics. These diagnostics ensure that the assumptions of the model have not been violated.

The following aspects are examined:

1. Non-linearity of data

2. Correlation of error terms

3. Non-constant variance

4. Outliers

5. High-leverage points

6. Collinearity

## Outliers